In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import zipfile
import shutil
import random

In [ ]:
# Unzipping Data in to the tmp folder

path = "/kaggle/input/dogs-vs-cats/train.zip"
zip_ref = zipfile.ZipFile(path)
zip_ref.extractall('/kaggle/tmp/PetImages/')

path = "/kaggle/input/dogs-vs-cats/test1.zip"
zip_ref = zipfile.ZipFile(path)
zip_ref.extractall('/kaggle/tmp/test_dataset/')



In [ ]:
# Dividing Cat and Dogs Images in separate folders

try:
    os.mkdir('/kaggle/tmp/PetImages/cats')
    os.mkdir('/kaggle/tmp/PetImages/dogs')
except OSError:
    pass

source = "/kaggle/tmp/PetImages/train"
dest_cats = "/kaggle/tmp/PetImages/cats"
dest_dogs = "/kaggle/tmp/PetImages/dogs"

for filename in os.listdir(source):
    if 'cat' in filename:
        shutil.copyfile(os.path.join(source, filename), os.path.join(dest_cats, filename))
    else:
        shutil.copyfile(os.path.join(source, filename), os.path.join(dest_dogs, filename))

print(len(os.listdir('/kaggle/tmp/PetImages/cats')))
print(len(os.listdir('/kaggle/tmp/PetImages/dogs')))


In [ ]:
# Creating Folders for Image Data Generator

# Creating Folders for Training Dataset
try:
    os.mkdir('/kaggle/tmp/cats-v-dogs')
    os.mkdir('/kaggle/tmp/cats-v-dogs/training_dataset')
    os.mkdir('/kaggle/tmp/cats-v-dogs/training_dataset/cats')
    os.mkdir('/kaggle/tmp/cats-v-dogs/training_dataset/dogs')
except OSError:
    pass
    

# Creating Folders for Validation Dataset
try:
    os.mkdir('/kaggle/tmp/cats-v-dogs/validation_dataset')
    os.mkdir('/kaggle/tmp/cats-v-dogs/validation_dataset/cats')
    os.mkdir('/kaggle/tmp/cats-v-dogs/validation_dataset/dogs')
except OSError:
    pass


In [ ]:
print(os.listdir('/kaggle/tmp/cats-v-dogs/training_dataset'))
print(os.listdir('/kaggle/tmp/cats-v-dogs/validation_dataset'))

In [ ]:
# Function for Spliting Data in to Training and Validation Datasets

def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
            
    training_length = int(len(files) * SPLIT_SIZE)
    validation_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-validation_length:]

    for filename in training_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING, filename)
        shutil.copyfile(this_file, destination)

    for filename in testing_set:
        this_file = os.path.join(SOURCE, filename)
        destination = os.path.join(VALIDATION, filename)
        shutil.copyfile(this_file, destination)

In [ ]:
# Calling split_data function

CAT_SOURCE = '/kaggle/tmp/PetImages/cats'
DOG_SOURCE = '/kaggle/tmp/PetImages/dogs'

CAT_TRAINING = "/kaggle/tmp/cats-v-dogs/training_dataset/cats"
DOG_TRAINING = "/kaggle/tmp/cats-v-dogs/training_dataset/dogs"

CAT_VALIDATION = "/kaggle/tmp/cats-v-dogs/validation_dataset/cats"
DOG_VALIDATION = "/kaggle/tmp/cats-v-dogs/validation_dataset/dogs"

SPLIT_SIZE = 0.9

split_data(CAT_SOURCE, CAT_TRAINING, CAT_VALIDATION, SPLIT_SIZE)
split_data(DOG_SOURCE, DOG_TRAINING, DOG_VALIDATION, SPLIT_SIZE)


In [ ]:
# Listing New Directory
validation_dir = '/kaggle/tmp/cats-v-dogs/training_dataset'
print("Training Tree")
print(os.listdir(validation_dir))
print("No. of Training Cats: ", len(os.listdir(CAT_TRAINING)))
print("No. of Training Dogs: ", len(os.listdir(DOG_TRAINING)))

validation_dir = '/kaggle/tmp/cats-v-dogs/validation_dataset'
print("\nValidation Tree")
print(os.listdir(validation_dir))
print("No. of Validation Cats: ", len(os.listdir(CAT_VALIDATION)))
print("No. of Validation Dogs: ", len(os.listdir(DOG_VALIDATION)))

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,)
train_generator = train_datagen.flow_from_directory(
    '/kaggle/tmp/cats-v-dogs/training_dataset',
    target_size=(150, 150),
    batch_size=50,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    '/kaggle/tmp/cats-v-dogs/validation_dataset',
    target_size=(150, 150),
    batch_size=50,
    class_mode='binary'
)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Downloading Local Weights File
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                               include_top=False,
                               weights=None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable=False
    
pre_trained_model.summary()


In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print("Last layer out put shape: ", last_layer.output_shape)
last_layer = last_layer.output


In [ ]:
# Flatten the output layer to 1 dimention
x = tf.keras.layers.Flatten()(last_layer)

x = tf.keras.layers.Dense(1024, activation='relu')(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)

x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(1, activation='relu')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator)

In [ ]:
# Ploting Training and Accuracy Graph

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

    
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")

plt.title('Training and validation loss')


In [ ]:

import numpy as np
from keras.preprocessing import image

test_data = '/kaggle/tmp/test_dataset/test1'
test_data_images = os.listdir(test_data)


predicted_value = []

for name in test_data_images[:100]:
    path = os.path.join(test_data, name)
    img = image.load_img(path, target_size=(150, 150))
    
    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    images = np.vstack([x])
    
    classes = model.predict(images, batch_size=10)
    
    if classes[0]>0:
        predicted_value.append('dog')
    else:
        predicted_value.append('cat')
    

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt



# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=16

next_test_pix = [os.path.join(test_data, fname) 
                for fname in test_data_images[ pic_index-16:pic_index] 
               ]

for i, img_path in enumerate(next_test_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)

    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.title(predicted_value[i])

plt.show()